# WFSS Box Extraction Example

This notebook demonstrates how to use the Generalized World Coordinate System (gWCS) in a Wide Field Slitless Spectrscopy (WFSS) observation to determine source locations and wavelengths. It shows how to calculate the location of a source in a WFSS observation given its location in a corresponding imaging observation, and how to calculate the wavelength at a particular pixel along an object's trace.

It then shows how to use the gWCS to perform a box extraction of a spectrum and translate the 1D spectrum into physical units.

In this example, we use exposures from JWST program 01076. We want to work on files that have full gWCS information in their headers, and that have had the flat field applied. We also need to run the flux calibration step of the pipeline in order to populate the name of the photom reference file in the header of the WFSS file (in the S_PHOTOM header keyword). This reference file will be used as part of the extraction process below. The photom step will not change the values of the science data in the WFSS exposure, because the observing mode (OBS_MODE header keyword) in the file is set to NRC_WFSS.

In order to accomplish this, the <b>assign_wcs</b>, <b>flat field</b>, and <b>photom</b> steps of the pipeline must be run on the data. Ordinarily this means we could simply download *_cal.fits files from MAST, and that is true for the imaging mode data used in this notebook. However as we show below, we want to apply the imaging mode flat field to the WFSS data. This means that we must download the *_rate.fits file, and manually run these pipeline steps on the data. For consistency, we do the same with the imaging mode data.

JWST detectors show little to no wavelength dependence in their flat-field, and just as is regularly done with HST WFSS data, in this example we have the pipeline apply the flat field for the direct cross filter to all the imaging as well as WFSS observations. We do not use a WFSS-specific flat field.

Once the data have been properly calibrated, the notebook uses the <b>grismconf</b> package to translate between source locations in the imaging and WFSS data, and calculate wavelengths associated with a given location in the WFSS data. <b>grismconf</b> also uses the flux calibration curve in the photom reference file for the grisms to translate the data from units of $DN/sec$ to $F_{lambda}$ units ($erg / sec / cm^2 / \overset{\circ}{A}$). <b>grismconf</b> will obtain the needed NIRCam WFSS configuration files from the Calibration Reference Data System (CRDS). Note that the photom step must be run on the data in order to obain the name of the approproate CRDS sesitivity file.

Note: At this stage, the important part of this is not the absolute accuracy of the WCS. Instead, we rely on accurate self-consistency between the imaging and the WFSS observations. 

Author: N. Pirzkal <br>
Date created: 24 Sept 2024

## Table of Contents
1. [Package Imports](#Package-Imports)
2. [Define Functions and Parameters](#Define-Functions-and-Parameters)
3. [Download Data](#Download-Data)
4. [Run Pipeline Steps](#Run-Pipeline-Steps)
5. [Basic Computation of WFSS Information](#Basic-Computation-of-WFSS-Information)
      * [Compute where light gets dispersed to](#Compute-where-light-gets-dispersed-to)
      * [Compute the spectral trace for a given object](#Compute-the-spectral-trace-for-a-given-object)
      * [Basic Box Extraction](#Basic-Box-Extraction)

## Package Imports

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from scipy.stats import sigmaclip

import grismconf
from jwst.assign_wcs import AssignWcsStep
from jwst.flatfield import FlatFieldStep
from jwst.photom import PhotomStep

## Set CRDS Path and Server

Before running the pipeline steps, we need to ensure our our CRDS environment is configured. This includes defining a CRDS cache directory in which to keep the reference files that will be used by the calibration pipeline.

If the root directory for the local CRDS cache has not already been set, it will be created in the home directory.

In [ ]:
# Check whether the local CRDS cache directory has been set.
# If not, set it to the user home directory
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')
# Check whether the CRDS server URL has been set.  If not, set it.
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Echo CRDS path and context in use
print('CRDS local filepath:', os.environ['CRDS_PATH'])
print('CRDS file server:', os.environ['CRDS_SERVER_URL'])

# import crds after setting up the required environment variables
from crds import client
if client.get_crds_server() != os.environ['CRDS_SERVER_URL']:
    client.set_crds_server('https://jwst-crds.stsci.edu')

## Define Functions and Parameters

Define a function to download a named file via the MAST API to the current directory. The function includes authentication logic, but the example in this notebook uses public data, so no MAST API token is required.

In [ ]:
def get_jwst_file(name, mast_api_token=None, overwrite=False):
    """Retrieve a JWST data file from MAST archive.
    
    Parameters
    ----------
    name : str
        Name of the file to download from MAST
        
    mast_api_token : str
        MAST API token. Required only for proprietary data
        
    overwrite : bool
        If True and the requested file already exists locally, the file will not be downloaded. IF False,
        the file will be downloaded
    """
    # If the file already exists locally, don't redownload it, unless the
    # user has set the overwrite keyword
    if os.path.isfile(name):
        if not overwrite:
            print(f'{name} already exists locally. Skipping download.')
            return
        else:
            print(f'{name} exists locally. Re-downloading.')

    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"
    params = dict(uri=f"mast:JWST/product/{name}")
    if mast_api_token:
        headers = dict(Authorization=f"token {mast_api_token}")
    else:
        headers = {}
    r = requests.get(mast_url, params=params, headers=headers, stream=True)
    r.raise_for_status()
    with open(name, "wb") as fobj:
        for chunk in r.iter_content(chunk_size=1024000):
            fobj.write(chunk)

Define a function that will run assign_wcs and flat fielding on an input rate file

In [ ]:
def run_pipeline_steps(filename):
    """Run the assign_wcs, flat field, and photom calibration steps on the given file.
    If the file contains WFSS data, trick the pipeline to use the imaging mode flat
    field reference file.
    
    Parameters
    ----------
    filename : str
        Name of the input file upon which the steps will be run
        
    Returns
    -------
    filename : str
        Name of the output file saved by the pipeline steps
        
    photom : jwst.datamodels.ImageModel
        Datamodel instance containing the calibrated data
    """
    assign_wcs = AssignWcsStep.call(filename)

    # In order to apply the imaging mode flat field reference file to the data,
    # we need to trick CRDS by temporarily changing the pupil value to be CLEAR
    reset_pupil = False
    if 'GRISM' in assign_wcs.meta.instrument.pupil:
        true_pupil = deepcopy(assign_wcs.meta.instrument.pupil)
        assign_wcs.meta.instrument.pupil = 'CLEAR'
        reset_pupil = True

    # Run the flat field step
    flat = FlatFieldStep.call(assign_wcs, save_results=True)
    
    # Run the photom step to populate the name of the WFSS sensitivity 
    photom = PhotomStep.call(flat, save_results=True)
    
    # Set the pupil back to the original value now that flat fielding is complete
    if reset_pupil:
        photom.meta.instrument.pupil = true_pupil
        photom.save(photom.meta.filename)
    
    # Return the name of the output file, as well as the datamodel
    return photom.meta.filename, photom

## Download Data

Download an example imaging mode rate file and corresponding WFSS mode rate file from MAST.

In [ ]:
# First, download the imaging and WFSS files from MAST
imaging_file = "jw01076103001_02102_00001_nrcalong_rate.fits"
wfss_file = "jw01076103001_02101_00001_nrcalong_rate.fits"
get_jwst_file(imaging_file)
get_jwst_file(wfss_file)

## Run Pipeline Steps

Run the assign_wcs, flat field, and photom calibration steps on both the imaging and WFSS files.

In [ ]:
# Run AssignWcsStep, FlatFieldStep, and PhotomStep on the imaging rate file
imaging_flat_file, imaging_data = run_pipeline_steps(imaging_file)

In [ ]:
# Run AssignWcsStep, FlatFieldStep, and PhotomStep on the WFSS rate file
wfss_flat_file, wfss_data = run_pipeline_steps(wfss_file)

## Basic Computation of WFSS Information

All computations for WFSS are performed in detector coordinate space. All of the characteristics of the dispersed traces, including any change in the relative positions and the global shape (e.g. curvature, offsets...) of the traces is handled using a series of straight forward equations. This is described in <a href="https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2017/WFC3-2017-01.pdf">ISR WFC3 2017-01</a>: "A more generalized coordinate transformation approach for grisms".
Here we assume that a source would be at the pixel coordinates of ($x$, $y$). The coordinate of a single pixel on on the dispersed trace for the same source is denoted as ($x_g$, $y_g$) and the relative position of this dispersed trace element is therefore offset (x$_g$-x, 
 y$_g$-y) pixels with respect to the position of the source. The functional relation between ($x$, $y$), ($x_g$, $y_g$) and the wavelength of the light $\lambda$, as well as their inverses are:

$$
\begin{align}
\delta x = x_g - x = f_x(x,y;t)\\
\delta y = y_g - y = f_y(x,y;t)\\
\lambda = f_\lambda(x,y;t)
\end{align}
$$

and 
$$
\begin{align}
t = f^{-1}_x(x,y;\delta x)\\
t = f^{-1}_y(x,y;\delta y)\\
t = f^{-1}_\lambda(x,y;\lambda)
\end{align}
$$

Note that these functions are parametrized with respect to the parameter $t$. This allows for some flexibility on the part of the calibration effort as $t$ can be defined somewhat arbitrarilly. In the case of the NIRCam grisms however, $t$ was chosen to be the $\delta x$ or $\delta y$, for the GRISMR and GRISMC, respectively since these grisms disperse light along the x-direction and y-direction, respectively. However, for additional convenience, the $t$ parameter is normalized to unity so that values of $t = 0$ and $t = 1$ correspond to the blue and red light edges of a dispersed spectrum.
Using the 6 equations above, one can relate any combination of ($x$,$y$), ($x'$,$y'$), $t$, and $\lambda$ values. The equations listed above are implemented as DISPX(), DISPY(), DISPL(), INVDISPX(), INVDISPY(), and INVDISPL() in the GRISMCONF package.

Now we will use the Grismconf package to retrieve information about the WFSS file. Note that we are using the output file from the calibration steps above.

In [ ]:
# This is the final output file from the pipeline call on the WFSS file above
wfss_file = "jw01076103001_02101_00001_nrcalong_photomstep.fits"

In [ ]:
# Load a WFSS configuration file to use in the example below.
C = grismconf.Config(wfss_file)

### Compute where light gets dispersed to

Here we show how to calculate the location of the point on the trace corresponding to a given wavelength for a source at a given detector location ($x$, $y$). For these calculations, we need only the WFSS file. The corresponding imaging mode file is not necessary.

In [ ]:
x = 1000  # Pixel x coordinate
y = 1000  # Pixel y coordinate

wavelength = 3.5  # wavelength, in microns

We want to compute $\hat x$, the amount of dispersion in a pixel for photons with a wavelength of $\lambda$. We first use the relation between $t$ and $\lambda$ and then the relation between $\hat x$ and $t$. This is done using INVDISPL() for order "+1" for an object at location ($x$, $y$):

Check which orders are available

In [ ]:
C.orders

Calculate $t$ for the given position and wavelength.

In [ ]:
t = C.INVDISPL("+1", x, y, wavelength)
print("t =", t)

We now can compute $\delta x$ and $\delta y$ using DISPX():

In [ ]:
𝛿x = C.DISPX("+1", x, y, t)
𝛿y = C.DISPY("+1", x, y, t)
print("𝛿x =", 𝛿x)
print("𝛿y =", 𝛿y)

The final pixel coordinates are therefore:

In [ ]:
xg = x + 𝛿x
yg = y + 𝛿y
print("Trace coordinates:", xg, yg)

Alternatively, we could compute the approximate wavelength of the light at a given position on the spectral trace. For example, we would like to compute the wavelength of a pixel that is at coordinates ($x_g$, $y_g$) for a 1st order spectrum of a source that is known to be at the coordinates ($x$, $y$). As this is a Grism R spectrum, we can use the relation between $\delta x$ and t and $\lambda$. 

In [ ]:
# Source is at the coordinates (1000, 1000) and we are looking at a pixel
# along the trace at pixel coordinate 1558
x = 1000
y = 1000

In [ ]:
t = C.INVDISPX("+1", x, y, xg-x)

In [ ]:
wavelength = C.DISPL("+1", x, y, t)
print(f"Wavelength = {wavelength} microns")

Here we see that we get back the 3.5 micron wavelength that we used as input when calculating $x_g$ and $y_g$ above.

### Compute the spectral trace for a given object

We can compute where we would expect the dispersed 1st order trace for a given object in a similar manner. We can use a series of $t$ values to cover the whole spectra trace (in this case the NIRCam calibration assumes $0<t<1$)

In [ ]:
x = 1000
y = 1000
ts = np.arange(0, 1, 0.01)

We can compute all of the corresponding trace coordinates and wavelengths (x$_g$, y$_g$, $\lambda$):

In [ ]:
xgs = C.DISPX("+1", x, y, ts) + x
ygs = C.DISPY("+1", x, y, ts) + y 
waves = C.DISPL("+1", x, y, ts)

Show an image of the location of the trace across the detector.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 2))
plt.scatter(xgs, ygs, c=waves)
plt.colorbar(label="Wavelength (μm)")
plt.grid()
plt.ylabel(r"y$_g$")
plt.xlabel(r"x$_g$")
plt.title("Trace location across the detector")

### Basic Box Extraction

A very basic "extraction" of a spectrum can be performed using the WFSS transformation listed above. Here we show how one could perfrom a basic "Box" extraction of a well-isolated object i.e. not contaminated by overlapping spectra from other sources in the field).

One of the key concepts is the virtual location of the source in the dispersed WFSS observation. We are not able to determine that location precisely using the WFSS data alone and must rely on additional information. The location of the source is usually measured in a direct image of the field. In cases where the imaging and WFSS data are undithered with respect to one another, the position of the source in the WFSS observation is the position of the same source in the undithered imaging. 

If dithering was performed, one can rely on the WCS to estimate the location of the source in imaging data that would have been observed at the same position and orientation as the WFSS observation. This is done by computing the observed RA and Dec of the source in the available imaging data, and then converting these newly computed celestrial coordinates back to a detector ($x$, $y$) location using a WCS from the dithered WFSS observation.

#### Examine a source in the data

Look at a source in the imaging data

In [ ]:
xd, yd = 1562, 696

In [ ]:
plt.imshow(imaging_data.data[yd - 20:yd + 20, xd - 20:xd + 20], origin="lower", vmin=0, vmax=10)
plt.xlabel('Column Number')
plt.ylabel('Row Number')

#### Calculate trace location

Now get the WCS from the imaging and WFSS files, so that we can perform coordinate transforms and calculate where the trace of this source lands in the WFSS observation.

In [ ]:
imaging_to_world = imaging_data.meta.wcs.get_transform('detector', 'world')

In [ ]:
wfss_to_pix = wfss_data.meta.wcs.get_transform('world', 'detector')

Translate the source's (x, y) location in the imaging mode data to RA, Dec, using the imaging mode WCS

In [ ]:
ra, dec = imaging_to_world(xd, yd)
print(ra, dec)

Now translate the RA, Dec to a location on the detector in the WFSS data, using the WCS from the WFSS file. Note that for this translation, the wavelength and order are required inputs, but they do not actually affect the calculation. We'll use a wavelength of 3.56 microns and an order of 1 in the cell below, but you can see that changing these values does not change the resulting x, y values.

In [ ]:
x, y, wav, ord = wfss_to_pix(ra, dec, 3.56, 1)
x, y, wav, ord

Compute an approximate bounding box for the spectrum of this source. We use the fact that calibration define the edges of the spectra at values of t=0 and t=1 in the dispersion direction

In [ ]:
# First calculate the distance from the nominal x, y location to the left and right
# edges of the box (i.e. where t = 0 and t = 1)
ts = np.array([0, 1])
dxs = C.DISPX("+1", x, y, ts)
dys = C.DISPY("+1", x, y, ts)
dxs, dys

In [ ]:
# Use the distances above to calculate the x location of the left and right edges of the box.
x_min = int(x + dxs[0])
x_max = int(x + dxs[1])

# Set the height of the box to be 50 pixels (25 pixels above and below the nominal location
y_min = int(y + dys[1] - 25)
y_max = int(y + dys[1] + 25)

x_min, x_max

Show an image of the box to be extracted.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 5))
ax.imshow(wfss_data.data[y_min:y_max, x_min:x_max], origin="lower", vmin=0.25, vmax=1)

ax.plot(x+dxs-x_min, y+dys-y_min)
ax.set_xlabel('Column Number')
ax.set_ylabel('Row Number')

The simplest extraction can be done under the assumption that the spectral trace is essentially linear and that the wavelength of the light falling on a pixel is only a function of the dispersion direction. As such, we can use the approximation that all pixels in a column correspond to the same wavelength. This is only appropriate for nearly flat dispersions and has often been assumed in the past. This also assumes that the field dependence of the dispersion is small within the area covered by a spectrum.

 #### Estimate Background

Here, for this simplest case, we estimate and subtract a simple background value from the data, taken to be the median value in the region contianing our spectrum. This very simple step illustrates the limits of simple box extraction as any such estimate will be biased by any other sources in the field, or any non-flatness in the structure of the dispersed background. 

In [ ]:
# We want to ignore all NaN pixels, so locate all non-NaN pixels
ok = np.isfinite(wfss_data.data[y_min:y_max, x_min:x_max])

# Sigma-clip the pixels in the box
clipped, low, upp = sigmaclip(wfss_data.data[y_min:y_max, x_min:x_max][ok], 1.5, 1.5)

# Calculate the median of the sigma-clipped pixels
med_bck = np.nanmedian(clipped)
print("Background level estimate:", med_bck)

In [ ]:
# Plot a histogram of the data, along with a line indicating the median level
plt.hist(np.ravel(wfss_data.data[y_min:y_max, x_min:x_max]), 100, range=(0, 1.5))
plt.axvline(med_bck, color='r')
plt.title('Histogram of pixel values in extraction box')
plt.xlabel('Signal Value (DN/sec)')
plt.ylabel('Occurrences')

#### Assign Wavelengths

Compute the wavelength everywhere on the array for this object. We use the full array here for simplicity and compute the distance in the x- direction (dispersion direction) between a pixel and the source.

In [ ]:
ys, xs = np.indices(np.shape(wfss_data.data))

Translate the x values to be distances from the nominal source location

In [ ]:
dxs = xs - x

We use the grismconf relations to compute the $t$ values everywhere and then compute the wavelengths, again everywhere.

In [ ]:
ts = C.INVDISPX("+1", x, y, dxs)
lams = C.DISPL("+1", x, y, ts)

Displaying the wavelength array on the same area on the detector where the spectrum is, we see what we now have a wavelength estimate for each element of the 2D WFSS data.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 3))
tt = ax.imshow(lams[y_min:y_max, x_min:x_max], origin="lower", vmin=2, vmax=4)
cbar = plt.colorbar(tt).ax.set_ylabel('Wavelength (microns)', rotation=270, fontsize=12, labelpad=25)
plt.title('Wavelength array covering all pixels of extraction box')
plt.xlabel('Column Number')
plt.ylabel('Row Number')

#### Box extraction

Creating a 1D spectrum from the data above is, in its simplest form, a matter of collapsing things along the cross dipsersion direction (y- direction). The 1D spectrum will be affected by bad pixels, any spectra contamination, as well as any error in the amount of background subtracted. These effects become increasingly important as the box size is increased. We also subtract the median background from each column.

In [ ]:
cs = np.nansum(wfss_data.data[y_min:y_max, x_min:x_max] - med_bck, axis=0)
plt.plot(cs)
plt.ylim(-5, 20)
plt.grid()
plt.xlabel('Column Number')
plt.ylabel('Signal (DN/sec)')
plt.title('Extracted 1D spectrum versus detector column')

#### Wavelength calibration

In this case, we can average the wavelength array in the cross dispersion direction (y-direction) to obtain a single wavelength vector that is appropriate for the spectum we just box extracted. Plotting the 1D count vector versus the wavelength vector thus created results in a wavelength calibrated 1D spectrum

In [ ]:
ws = np.nanmean(lams[y_min:y_max, x_min:x_max], axis=0)
plt.plot(ws, cs)
plt.ylim(-5, 20)
plt.grid()
plt.xlabel('Wavelength (microns)')
plt.ylabel('Signal (DN/sec)')
plt.title('Extracted 1D spectrum versus wavelength')

#### Converting to physical units

Here we use information contained in grismconf to convert our 1D spectrum, which is in units of DN/sec (per bin, or pixel in this case since there is no resampling of the data) into units of $erg / sec / cm^2 / \overset{\circ}{A}$. The grism inverse sensitivity curve in grismconf is in units of DN/sec (per pixel) per $erg / sec / cm^2 / \overset{\circ}{A}$. As the sensitivity is defined per Angstrom while our 1D spectrum is extracted per pixel, we need to account for this when applying the sensitivity. In most WFSS data the variation of the pixel size in wavelength units is small but this can easily be accounted for by computing the size of each pixel in wavelength units (Angstrom) as we show below.

In [ ]:
# Plot the inverse sensitivity curve
tws = np.arange(2.9, 4.2, 0.01)
plt.plot(tws, C.SENS["+1"](tws))
plt.grid()
plt.xlabel("Wavelength (micron)")
plt.ylabel("Inverse Sensitivity")
plt.title("Inverse sensitivity curve")

Calculate the size of each element in our 1D spectrum, in Angstrom

In [ ]:
# The value of 10000 comes from the NIRCam dispersion value of ~10A per pixel
dws = (ws[1:] - ws[:-1]) * 10000

Apply the sensitivity curve by dividing the 1D counts by the sensitivity and by the size of each bin in Angstrom to produce flux values in units of $erg / sec / cm^2 / \overset{\circ}{A}$

In [ ]:
fs = cs[1:] / C.SENS["+1"](ws[1:]) / dws

Plotting the resulting flux array versus our wavelength scale shows the fully calbirated 1D spectrum. One caveat of the conversion from DN/sec per pixel to $erg / sec / cm^2 / \overset{\circ}{A}$ is the well known edge effect caused by dividing counts values that are zero or close to zero by the sensitivity. As this method does not account for the broadening of the spectra by the footprint of the object in the dispersion direction, this effect is worse for extended sources. This effect can only be mitigated by more forward modeling extraction methods.

In [ ]:
plt.plot(ws[1:], fs)
plt.xlabel("Wavelength (micron)")
plt.ylabel("Signal ($erg/s/cm^2/A$)")
plt.title("Extracted 1D spectrum in $F_{lambda}$ units")

Concentrating on regions off the spectral edges reveals the spectrum of our source. One of the many drawbacks of this simplistic box extraction method is the impact of bad pixels and any residual cosmic rays. These can cause significnat peaks in the spectra, which can be confused for emission lines.

In [ ]:
plt.plot(ws[1:], fs)
plt.xlim(3.1, 4.)
plt.ylim(0, 0.5e-16)
plt.xlabel("Wavelength (micron)")
plt.ylabel("Signal ($erg/s/cm^2/A$)")
plt.title("Extracted 1D spectrum in $F_{lambda}$ units")